# STEP 3 - Take synthetic seismograms for the quakes in the catalog

The script uses the catalogs that have been already generated (as GCMT solution files) in the previous step, in order to download synthetic data from the IRIS instaseis databases. The data are obtained for stations with epicentral distances 5, 10, and 15 degrees, with backazimuths of 0, 45, and 90 degrees. It saves the displacement seismograms for Z, R, and T components in a unique mseed file.

© Foivos Karakostas - UCL, 2024

-----------------------------------------

##### User inputs

In [52]:
synt_model = 'iasp91_2s'
taup_model = 'iasp91';
threshold = 45
components = 'ZNE'
units = 'displacement'
duration = 600

-----------------------------------------

Import the necessary packages

In [53]:
import obspy
from obspy import read_events
from obspy.clients.syngine import Client
from obspy import UTCDateTime
import os
from geopy.distance import distance
from geopy import Point
import pandas as pd
from matplotlib import pyplot as pplot

In [54]:
def generate_synthetics(event_lat, event_lon, mt, model,source_depth_in_km,origin_time,components,units,duration,directory):
    
    mrr, mtt, mpp, mrt, mrp, mtp = mt
    
    event_location = Point(event_lat, event_lon)
    stations = []
    network = 'SY'
    
    distances = list(range(5, 16, 5))
    backazimuths = list(range(0, 91, 45))
    
    for dist in(distances):
        for baz in(backazimuths):
            station = 'SD' + str(dist).zfill(3) + 'B' + str(baz).zfill(3)
            event_location = Point(event_lat, event_lon)
            station_location = distance(kilometers=dist * 111.32).destination(event_location, baz)
            stations.append({
                "Epicentral_Distance": dist,
                "Azimuth": baz,
                "Station_Latitude": station_location.latitude,
                "Station_Longitude": station_location.longitude
            })
            
            depthinm = source_depth_in_km * 1000
            etime = origin_time + duration
            
            filename = network + '.' + station + '.mseed'
            filepath = directory + '/' + filename
            
            client = Client()
            
            st = client.get_waveforms(model=model, 
                                      receiverlatitude=station_location.latitude, 
                                      receiverlongitude=station_location.longitude,
                                      sourcelatitude=event_location.latitude, 
                                      sourcelongitude=event_location.longitude, 
                                      sourcedepthinmeters=depthinm, 
                                      sourcemomenttensor=[mrr, mtt, mpp, mrt, mrp, mtp], 
                                      origintime=origin_time, 
                                      starttime=origin_time, 
                                      endtime=etime, 
                                      components=components, 
                                      units=units, 
                                      format='miniseed',
                                      filename=filepath
                                      )
            

In [55]:
SyntDir = 'Synthetics' + str(threshold)
os.makedirs(SyntDir, exist_ok=True)
client = Client()

Start with the Strike-slip Catalog

In [ ]:
cmtsismofilename1 = 'Catalogs/CMTSOLUTIONS_Strikeslip_' + str(threshold)
syntsdirectory = SyntDir + '/Strikeslip'


os.makedirs(syntsdirectory, exist_ok=True)

catcmt = read_events(cmtsismofilename1)
for e in range (0, len(catcmt)):
    moment_tensor = catcmt[e].focal_mechanisms[0].moment_tensor;
    quake_name = catcmt[0].event_descriptions[0].text
    eventlon = catcmt[e].origins[0].longitude;
    eventlat = catcmt[e].origins[0].latitude;
    eventdepth = catcmt[e].origins[0].depth/1000;
    starttime = catcmt[e].origins[0].time;
    region = catcmt[e].origins[0].region;
    eventmag = catcmt[e].magnitudes[0].mag;
    mrr = moment_tensor.tensor.m_rr;
    mtt = moment_tensor.tensor.m_tt;
    mpp = moment_tensor.tensor.m_pp;
    mrt = moment_tensor.tensor.m_rt;
    mrp = moment_tensor.tensor.m_rp;
    mtp = moment_tensor.tensor.m_tp;
    mt = [mrr, mtt, mpp, mrt, mrp, mtp];
    client = Client("IRIS")
    eventid = 'M' + str(starttime.year).zfill(4) + str(starttime.month).zfill(2) + str(starttime.day).zfill(2) + str(starttime.hour).zfill(2) + str(starttime.minute).zfill(2) + str(starttime.second).zfill(2)
    eventdirectory = syntsdirectory + '/' + eventid
    os.makedirs(eventdirectory, exist_ok=True)
    
    generate_synthetics(eventlat, eventlon, mt, synt_model,eventdepth,starttime,components,units,duration,eventdirectory)

In [ ]:
cmtsismofilename1 = 'Catalogs/CMTSOLUTIONS_Normal_' + str(threshold)
syntsdirectory = SyntDir + '/Normal'


os.makedirs(syntsdirectory, exist_ok=True)

catcmt = read_events(cmtsismofilename1)
for e in range (0, len(catcmt)):
    moment_tensor = catcmt[e].focal_mechanisms[0].moment_tensor;
    quake_name = catcmt[0].event_descriptions[0].text
    eventlon = catcmt[e].origins[0].longitude;
    eventlat = catcmt[e].origins[0].latitude;
    eventdepth = catcmt[e].origins[0].depth/1000;
    starttime = catcmt[e].origins[0].time;
    region = catcmt[e].origins[0].region;
    eventmag = catcmt[e].magnitudes[0].mag;
    mrr = moment_tensor.tensor.m_rr;
    mtt = moment_tensor.tensor.m_tt;
    mpp = moment_tensor.tensor.m_pp;
    mrt = moment_tensor.tensor.m_rt;
    mrp = moment_tensor.tensor.m_rp;
    mtp = moment_tensor.tensor.m_tp;
    mt = [mrr, mtt, mpp, mrt, mrp, mtp];
    client = Client("IRIS")
    eventid = 'M' + str(starttime.year).zfill(4) + str(starttime.month).zfill(2) + str(starttime.day).zfill(2) + str(starttime.hour).zfill(2) + str(starttime.minute).zfill(2) + str(starttime.second).zfill(2)
    eventdirectory = syntsdirectory + '/' + eventid
    os.makedirs(eventdirectory, exist_ok=True)
    
    generate_synthetics(eventlat, eventlon, mt, synt_model,eventdepth,starttime,components,units,duration,eventdirectory)

In [ ]:
cmtsismofilename1 = 'Catalogs/CMTSOLUTIONS_Reverse_' + str(threshold)
syntsdirectory = SyntDir + '/Reverse'


os.makedirs(syntsdirectory, exist_ok=True)

catcmt = read_events(cmtsismofilename1)
for e in range (0, len(catcmt)):
    moment_tensor = catcmt[e].focal_mechanisms[0].moment_tensor;
    quake_name = catcmt[0].event_descriptions[0].text
    eventlon = catcmt[e].origins[0].longitude;
    eventlat = catcmt[e].origins[0].latitude;
    eventdepth = catcmt[e].origins[0].depth/1000;
    starttime = catcmt[e].origins[0].time;
    region = catcmt[e].origins[0].region;
    eventmag = catcmt[e].magnitudes[0].mag;
    mrr = moment_tensor.tensor.m_rr;
    mtt = moment_tensor.tensor.m_tt;
    mpp = moment_tensor.tensor.m_pp;
    mrt = moment_tensor.tensor.m_rt;
    mrp = moment_tensor.tensor.m_rp;
    mtp = moment_tensor.tensor.m_tp;
    mt = [mrr, mtt, mpp, mrt, mrp, mtp];
    client = Client("IRIS")
    eventid = 'M' + str(starttime.year).zfill(4) + str(starttime.month).zfill(2) + str(starttime.day).zfill(2) + str(starttime.hour).zfill(2) + str(starttime.minute).zfill(2) + str(starttime.second).zfill(2)
    eventdirectory = syntsdirectory + '/' + eventid
    os.makedirs(eventdirectory, exist_ok=True)
    
    generate_synthetics(eventlat, eventlon, mt, synt_model,eventdepth,starttime,components,units,duration,eventdirectory)

In [ ]:
cmtsismofilename1 = 'Catalogs/CMTSOLUTIONS_Unclassified_' + str(threshold)
syntsdirectory = SyntDir + '/Unclassified'


os.makedirs(syntsdirectory, exist_ok=True)

catcmt = read_events(cmtsismofilename1)
for e in range (0, len(catcmt)):
    moment_tensor = catcmt[e].focal_mechanisms[0].moment_tensor;
    quake_name = catcmt[0].event_descriptions[0].text
    eventlon = catcmt[e].origins[0].longitude;
    eventlat = catcmt[e].origins[0].latitude;
    eventdepth = catcmt[e].origins[0].depth/1000;
    starttime = catcmt[e].origins[0].time;
    region = catcmt[e].origins[0].region;
    eventmag = catcmt[e].magnitudes[0].mag;
    mrr = moment_tensor.tensor.m_rr;
    mtt = moment_tensor.tensor.m_tt;
    mpp = moment_tensor.tensor.m_pp;
    mrt = moment_tensor.tensor.m_rt;
    mrp = moment_tensor.tensor.m_rp;
    mtp = moment_tensor.tensor.m_tp;
    mt = [mrr, mtt, mpp, mrt, mrp, mtp];
    client = Client("IRIS")
    eventid = 'M' + str(starttime.year).zfill(4) + str(starttime.month).zfill(2) + str(starttime.day).zfill(2) + str(starttime.hour).zfill(2) + str(starttime.minute).zfill(2) + str(starttime.second).zfill(2)
    eventdirectory = syntsdirectory + '/' + eventid
    os.makedirs(eventdirectory, exist_ok=True)
    
    generate_synthetics(eventlat, eventlon, mt, synt_model,eventdepth,starttime,components,units,duration,eventdirectory)